In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

from skimage import color, feature, exposure

import random
import cv2

In [ ]:
resize_size = 300
batch_size = 45

In [ ]:
# Augmentation is not applied for Test/Validation data
transform_valid = transforms.Compose([
    transforms.Resize((resize_size,resize_size)),
    
    transforms.ToTensor(),
    #transforms.ToPILImage(), # the transform usually work with PIL images
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

In [ ]:
data_root = 'testdata'
custom_dataset = ImageFolder(root=data_root, transform=transform_valid)

# specify classes
classes = ('cherry','strawberry', 'tomato')

## 1. Load the CNN model

In [85]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjust input size based on your images
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 3)# num_classes is the number of classes in your dataset

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)  # "fc" stands for fully connected, i.e. a nn.Linear layer
        x = self.relu3(x)
        x = self.fc2(x)
        return x

In [86]:
PATH = './model.pth'

model = nn.Sequential(
    SimpleCNN(),
    nn.Sigmoid()
)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

## 2. Evaluate the model

In [87]:
# Evaluation on the test set
model.eval()
all_predictions = []
all_labels = []

In [88]:
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [89]:
# Calculate accuracy on the test set
accuracy = accuracy_score(all_labels, all_predictions)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 35.33%
